In [32]:
import tensorflow as tf

# Allow GPU memory growth
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU available.")

# Initialize GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

1 Physical GPUs, 1 Logical GPUs
Num GPUs Available:  1


In [33]:
import os
import cv2
import numpy as np
import tensorflow as tf
from skimage.transform import rotate

# Rutas a las carpetas de entrenamiento, prueba y validación para imágenes y máscaras
train_frames_dir = '../Frames/TRAIN'
train_Landmarks_dir = './Landmarks/TRAIN'
test_frames_dir = '../Frames/TEST'
test_Landmarks_dir = './Landmarks/TEST'
val_frames_dir = '../Frames/VAL'
val_Landmarks_dir = './Landmarks/VAL'

# Funciones para cargar y preprocesar imagen y máscara y convertirlo a solo un canal
def load_image(file_path):
    image = cv2.imread(file_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = image / 255.0
    return image

def load_landmarks(file_path):
    images = []
    for filename in os.listdir(file_path):
        image=cv2.imread(os.path.join(file_path, filename))
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        images.append(grayscale_image)

    stacked_image = np.stack(images, axis=-1)  # axis=-1 indica la dimensión de los canales
    stacked_image = stacked_image / 255.0
    return stacked_image

# Data augmentation: Rotations and Flips
def augment_data(images, landmarks):
    augmented_images = []
    augmented_masks = []
    for image, mask in zip(images, landmarks):
        augmented_images.append(image)
        augmented_masks.append(landmarks)
        # # 20 degree rotations
        # augmented_images.append(rotate(image, angle=20, reshape=False))
        # augmented_masks.append(rotate(landmarks, angle=20, reshape=False))
        # augmented_images.append(rotate(image, angle=-20, reshape=False))
        # augmented_masks.append(rotate(landmarks, angle=-20, reshape=False))

        # Flips
        augmented_images.append(np.fliplr(image))
        augmented_masks.append(np.fliplr(landmarks))


    return np.array(augmented_images), np.array(augmented_masks)

train_image_files = [os.path.join(train_frames_dir, filename) for filename in os.listdir(train_frames_dir)]
train_Landmarks_files = [os.path.join(train_Landmarks_dir, filename) for filename in os.listdir(train_Landmarks_dir)]
test_image_files = [os.path.join(test_frames_dir, filename) for filename in os.listdir(test_frames_dir)]
test_Landmarks_files = [os.path.join(test_Landmarks_dir, filename) for filename in os.listdir(test_Landmarks_dir)]
val_image_files = [os.path.join(val_frames_dir, filename) for filename in os.listdir(val_frames_dir)]
val_Landmarks_files = [os.path.join(val_Landmarks_dir, filename) for filename in os.listdir(val_Landmarks_dir)]

test_images = [load_image(file) for file in test_image_files]
test_Landmarks = [load_landmarks(file) for file in test_Landmarks_files]

val_images = [load_image(file) for file in val_image_files]
val_Landmarks = [load_landmarks(file) for file in val_Landmarks_files]

test_images = np.array(test_images)
test_Landmarks = np.array(test_Landmarks)
val_images = np.array(val_images)
val_Landmarks = np.array(val_Landmarks)

In [34]:
# Agremos el path de la carpeta de modelos para poder importar el modelo
import sys
sys.path.append(r'R:\Codes\Reto\Modelos')

In [35]:
# Llamamos a la funcion del modelo U-Net desde un archivo externo
from model2 import unet_model

# Crear el modelo U-Net
model = unet_model(input_shape=(112, 112, 1), n_classes=7, kernel_out=1, activation='linear')

In [36]:
import tensorflow as tf
from tensorflow.keras import backend as K

def mean_squared_error_landmarks(y_true, y_pred):
    """
    Mean Squared Error loss function for 7 landmarks.
    Assumes y_true and y_pred are of shape (batch_size, 112, 112, 7).
    """
    mse = K.mean(K.square(y_true - y_pred), axis=-1)
    return mse

def accuracy_landmarks(y_true, y_pred):
    """
    Accuracy metric for 7 landmarks.
    Assumes y_true and y_pred are of shape (batch_size, 112, 112, 7).
    """
    # Extract the landmark values from y_true and y_pred
    y_true_landmarks = y_true[..., :7]
    y_pred_landmarks = y_pred[..., :7]

    # Calculate the element-wise absolute difference
    abs_diff = K.abs(y_true_landmarks - y_pred_landmarks)

    # Create a mask where each element is 1 if the absolute difference is less than a threshold, otherwise 0
    mask = K.cast(K.less(abs_diff, 0.5), dtype=tf.float32)

    # Calculate the accuracy for each landmark
    landmark_accuracy = K.mean(mask, axis=-1)

    # Overall accuracy is the mean accuracy across all landmarks
    overall_accuracy = K.mean(landmark_accuracy)

    return overall_accuracy

In [37]:
import tensorflow as tf
import numpy as np

# Early stopping and model checkpoint callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10, mode='min', monitor='val_loss'),
    tf.keras.callbacks.ModelCheckpoint(filepath='best_model.h5', save_best_only=True, monitor='val_loss', mode='min')
]


BATCH_SIZE = 8

with tf.device('CPU'):
    val_dataset = tf.data.Dataset.from_tensor_slices((val_images, val_Landmarks))
    val_dataset = val_dataset.batch(BATCH_SIZE)

    test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_Landmarks))
    test_dataset = test_dataset.batch(BATCH_SIZE)

# Compile the model outside the loop
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error', accuracy_landmarks])

for i in range(0, len(train_image_files), BATCH_SIZE):
    batch_image_paths = train_image_files[i:i+BATCH_SIZE]
    batch_landmark_paths = train_Landmarks_files[i:i+BATCH_SIZE]

    # Load and pre-process the images and landmarks
    batch_images = [load_image(file) for file in batch_image_paths]
    batch_landmarks = [load_landmarks(file) for file in batch_landmark_paths]

    # Data augmentation
    batch_images_aug, batch_landmarks_aug = augment_data(batch_images, batch_landmarks)

    # Convert the list of images and landmarks to a numpy array
    batch_images_aug = np.array(batch_images_aug)
    batch_landmarks_aug = np.array(batch_landmarks_aug)

    with tf.device('CPU'):
        train_dataset = tf.data.Dataset.from_tensor_slices((batch_images_aug, batch_landmarks_aug))
        train_dataset = train_dataset.batch(BATCH_SIZE)

    # Train the model for one epoch
    history = model.fit(train_dataset, epochs=100, validation_data=val_dataset, callbacks=callbacks)


Epoch 1/100


ResourceExhaustedError: Graph execution error:

Detected at node 'U-Net_2/batch_normalization_89/FusedBatchNormV3' defined at (most recent call last):
    File "c:\Users\luisr\anaconda3\envs\tf\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\luisr\anaconda3\envs\tf\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "c:\Users\luisr\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\luisr\anaconda3\envs\tf\lib\site-packages\traitlets\config\application.py", line 1053, in launch_instance
      app.start()
    File "c:\Users\luisr\anaconda3\envs\tf\lib\site-packages\ipykernel\kernelapp.py", line 737, in start
      self.io_loop.start()
    File "c:\Users\luisr\anaconda3\envs\tf\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\luisr\anaconda3\env

In [ ]:
evaluation = model.evaluate(test_dataset)

In [ ]:
# Save model history information to a json file
import json
import time
import matplotlib.pyplot as plt

# Generate a folder name
name_folder = 'landmark_aug_{}'.format(time.strftime("%Y%m%d-%H%M%S"))
path = '../Pruebas/{}/'.format(name_folder)
os.mkdir(path)

# Save history to a json file
path_json = '{}history.json'.format(path)

with open(path_json, 'w') as fp:
    json.dump(history.history, fp)

# Save model
path_model = '{}model.h5'.format(path)
model.save(path_model)

# Save model summary to a txt file
path_summary = '{}summary.txt'.format(path)
with open(path_summary, 'w') as fp:
    model.summary(print_fn=lambda x: fp.write(x + '\n'))

# Save model metrics to a txt file
path_metrics = '{}metrics.txt'.format(path)
with open(path_metrics, 'w') as fp:
    fp.write('Test loss: {}\n'.format(evaluation[0]))
    fp.write('Test accuracy_landmarks: {}\n'.format(evaluation[1]))
    fp.write('Model optimizer: {}\n'.format(model.optimizer.__class__.__name__))
    fp.write('Model activation: {}\n'.format(model.layers[-1].activation.__name__))
    fp.write('Model learning rate: {}\n'.format(model.optimizer.lr.numpy()))
    fp.write('Model loss method: {}\n'.format(loss_method))

# Generate a folder inside the folder of the model for the images
path_images = '{}images/'.format(path)
os.mkdir(path_images)

# Generate a folder inside the folder of the images for plot images
path_plot_images = '{}plot_images/'.format(path_images)
os.mkdir(path_plot_images)

# Generate a folder inside the folder of the images for prediction images
path_test_images = '{}predicitons/'.format(path_images)
os.mkdir(path_test_images)

# Save prediction images
# Función para visualizar una muestra de imágenes y sus máscaras de segmentación predichas
def visualize_landmarks(images, landmarks, predictions):
    num_images = len(images)

    for i in range(num_images):  # Renamed the inner loop variable to 'j'
        # Set up the subplots
        image = images[i].reshape((112, 112))
        prediction = predictions[i].reshape((112, 112, 7)) * 255.0
        landmark = landmarks[i] * 255.0  # No need to reshape
        fig, axs = plt.subplots(2, 9, figsize=(15, 5))

        # Display the test image spanning the first two columns of both rows
        axs[0, 0].imshow(image, cmap='grey')
        axs[0, 0].axis('off')
        axs[0, 0].set_title('Test Image') 
        axs[1, 0].axis('off')

        # Combine all channels of prediction_landmarks by taking the mean
        combined_prediction = np.mean(prediction, axis=-1)

        # CObine all channels of ground_truth_landmarks by taking the mean
        combined_ground_truth = np.mean(landmark, axis=-1)

        # Display the predicted and ground truth landmarks in the first column of the second row all in one image
        axs[0, 1].imshow(combined_prediction, cmap='grey')
        axs[0, 1].set_title('Predicted')
        axs[0, 1].axis('off')
        axs[1, 1].imshow(combined_ground_truth, cmap='grey')
        axs[1, 1].set_title('Ground Truth')
        axs[1, 1].axis('off')

        # Display the predicted landmarks in the first row
        for k in range(7):  # Renamed the loop variable to 'k'
            axs[0, k+2].imshow(prediction[ :, :, k], cmap='grey')
            axs[0, k+2].axis('off')
            axs[0, k+2].set_title('Predicted ' + str(k + 1))

        # Display the ground truth landmarks in the second row
        for k in range(7):  # Renamed the loop variable to 'k'
            axs[1, k+2].imshow(landmark[ :, :, k], cmap='grey')
            axs[1, k+2].axis('off')
            axs[1, k+2].set_title('Ground Truth ' + str(k + 1))

        plt.savefig('{}{}.png'.format(path_test_images, i))


# Supongamos que tienes un conjunto de datos de prueba con imágenes y máscaras
# Puedes utilizar el modelo para obtener las máscaras predichas en el conjunto de prueba
predictions = model.predict(test_dataset)

# Elije algunas muestras aleatorias para visualizar
num_samples_to_visualize = 5
sample_indices = np.random.choice(len(test_images[:10]), num_samples_to_visualize, replace=False)

sample_images = [test_images[i] for i in sample_indices]
sample_landmarks = [test_Landmarks[i] for i in sample_indices]
sample_predictions = [predictions[i] for i in sample_indices]

# Llama a la función para visualizar las imágenes y máscaras
visualize_landmarks(sample_images, sample_landmarks, sample_predictions)

# Save plot images
plt.figure()
plt.plot(history.history['accuracy_landmarks'])
plt.plot(history.history['val_accuracy_landmarks'])
plt.title('Model accuracy_landmarks')
plt.ylabel('accuracy_landmarks')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('{}{}.png'.format(path_plot_images, 'accuracy_landmarks'))

plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.savefig('{}loss.png'.format(path_plot_images))

In [ ]:
# Write a txt file with the augmentation information
path_aug = '{}aug.txt'.format(path)

with open(path_aug, 'w') as fp:
    # Write the technique used
    fp.write('Data augmentation: Flip Left Right\n')
    # fp.write('Data augmentation: Rotations 20 degrees\n')
    # fp.write('Data augmentation: Rotations -20 degrees\n')

In [ ]:
from skimage.metrics import structural_similarity as ssim

#TODO: Investigate what the ssim function does
def calculate_ssim(image1, image2):
    # Calculate the Structural Similarity Index (SSI)
    similarity_index, _ = ssim(image1, image2, full=True, data_range=image1.max() - image1.min())

    # The SSI ranges from -1 to 1, where 1 indicates a perfect match
    # We normalize it to the range [0, 1] to represent percentage similarity
    percentage_similarity = (similarity_index + 1) / 2 * 100

    return percentage_similarity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Calculate the mean Structural Similarity Index (SSI) for the landmarks in the test set
mean_ssim_landmark = 0
for i in range(len(test_Landmarks)):
    landmark1 = np.mean(test_Landmarks[i], axis=-1)
    landmark2 = np.mean(predictions[i], axis=-1)
    mean_ssim_landmark += calculate_ssim(landmark1, landmark2)

mean_ssim_landmark /= len(test_Landmarks)

# Calculate the mean absolute error for the landmarks in the test set
mean_abs_error_landmark = 0
for i in range(len(test_Landmarks)):
    landmark1 = np.mean(test_Landmarks[i], axis=-1)
    landmark2 = np.mean(predictions[i], axis=-1)
    mean_abs_error_landmark += np.mean(np.abs(landmark1 - landmark2))

mean_abs_error_landmark /= len(test_Landmarks)

# Save the mean SSI and mean absolute error to a file
f = open(path + "/results.txt", "a")
f.write("\n")
f.write("Mean SSI for Landmarks: " + str(mean_ssim_landmark) + "\n")
f.write("Mean absolute error for Landmarks: " + str(mean_abs_error_landmark) + "\n")
f.close()